In [ ]:
from pde import *
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm
import os
import json

This function lets me match the parameters to output in case I want to go back to a previous state.

In [ ]:
def save_params(params, dir):
    with open(dir, "w") as file:
        json.dump(params, file)

This is the class and code I am using to generate the model.

In [ ]:
# https://py-pde.readthedocs.io/en/latest/examples_gallery/pde_coupled.html#sphx-glr-examples-gallery-pde-coupled-py

class FitzhughNagumoPDE(PDEBase):
    """FitzHugh–Nagumo model with diffusive coupling"""

    def __init__(
        self,
        stimulus=0.5,
        τ=10,
        a=0,
        b=0,
        D=1,
        w_h=1,
        w_l=0.5,
        bc="auto_periodic_neumann"
    ):
        self.bc = bc
        self.stimulus = stimulus
        self.τ = τ
        self.a = a
        self.b = b
        self.D = D
        self.w_h = w_h
        self.w_l = w_l

    def evolution_rate(self, state, t=0):
        v, w = state  # membrane potential and recovery variable

        v_t = self.D * v.laplace(bc=self.bc) + self.stimulus + (v - v**3 / 3 - w)# * self.τ
        w_t = (v + self.a - self.b * w) / self.τ * ((self.w_h - self.w_l) / (1 + np.exp(-4 * v)) + self.w_l)

        return FieldCollection([v_t, w_t])

if __name__ == "__main__":
    # grid = UnitGrid([32, 32])
    grid = CartesianGrid([[-1, 1], [-1, 1]], [40, 40])
    # state = FieldCollection.scalar_random_uniform(2, grid)
    dx = 1/40
    origin = np.array([20, 20])
    data = np.zeros(shape=(2, 40, 40))
    # data[:, 9, 10] = data[:, 10, 9] = data[:, 11, 10] = data[:, 10, 11] = -0.5
    # data[:, 10, 10] = -1
    data[0, 20, 20] = -1.0367
    data[1, 20, 20] = -0.6656

    # Cos init
    # for i in range(40):
    #     for j in range(40):
    #         pos = np.array([i, j]) - origin
    #         pos = pos.astype(np.float64)
    #         pos *= dx
    #         r = np.sqrt(np.dot(pos, pos))
    #         data[0, i, j] = np.cos(r * 10 * np.pi)
    #         data[1, i, j] = np.cos(r * 10 * np.pi)


    v_state = ScalarField(grid, data[0])
    w_state = ScalarField(grid, data[1])
    state = FieldCollection((v_state, w_state))

    # Good waves #1
    # params = {
    #     "a": 0.7,
    #     "b": 0.5,
    #     "τ": 1/0.85,
    #     "stimulus": 1,
    #     "D": 0.0001,
    #     "w_h": 0.6,
    #     "w_l": 0.4,
    # }

    
    # N = 128
    # T = 1000
    # t0 = 0
    # dt = 0.1
    # s = 0.02 # 0.02 # 0.10
    # D = 1.0
    # a = 0.5
    # b = 0.7
    # c = 0.3
    # I = 1.0 # 1.0 # 0.5 # 1.0
    # Other params
    params = {
        "stimulus": 0,
        "a": 0.5,
        "b": 0.7,
        "τ": 1/0.3,
        "w_h": 1,
        "w_l": 1,
        "D": 1
    }

    # Chick
    # params = {
    #     "a": 0.7,
    #     "b": 0.5,
    #     "τ": 1/0.44,
    #     "stimulus": 0,
    #     "w_h": 0.6,
    #     "w_l": 0.4,
    #     "D": 0.001
    # }
    memory_storage = MemoryStorage()
    eq = FitzhughNagumoPDE(**params)
    result = eq.solve(state, t_range=100, dt=0.01, tracker=[memory_storage.tracker(0.1)])
    result.plot()
    plot_kymographs(memory_storage)
    data = memory_storage.data
    data = np.array(data)
    vs = data[:, 0, :]
    ws = data[:, 1, :]

    save_images = False
    if save_images:
        for a in tqdm(range(0, 1000, 20)):
            fig = plt.figure()
            fig.set_size_inches((8, 3))
            axs = fig.subplots(1, 2, squeeze=False)
            axes_image = axs[0, 0].imshow(vs[a], vmin=vmin, vmax=vmax)
            plt.colorbar(axes_image, ax=axs[0, 0])
            axes_image = axs[0, 1].imshow(ws[a], vmin=vmin, vmax=vmax)
            plt.colorbar(axes_image, ax=axs[0, 1])
            plt.savefig(f"./fn_images/{a}.png")
            plt.close(fig)
    print("Done!")
    


This lets me visualize the initial state in real numbers. (Useful when the state grows exponentially.)

In [ ]:
import pandas as pd
v_state = ScalarField(grid, data[0, 0])
w_state = ScalarField(grid, data[1, 0])
D = 1
bc = "auto_periodic_neumann"
τ = 1/0.44
stimulus = 0
v_t = D * v_state.laplace(bc=bc)# + τ * (v_state - v_state**3 / 3 - w_state) + stimulus
v_t.data
df = pd.DataFrame(v_t.data)
df.style


This cell plots the value and Laplacian at the location of the initial perterbation, so I can easily tell if the root is oscillating (value plot) and if the waves are actually going out (Laplace plot). I am hoping that both plots show oscillation.

In [ ]:
top_v = vs[:, 20, 20]
top_w = ws[:, 20, 20]
bc = "auto_periodic_neumann"
lap_v = np.empty(len(top_v))
lap_w = np.empty(len(top_w))
for a in range(len(top_v)):
    lap = ScalarField(grid, vs[a]).laplace(bc=bc)
    lap_v[a] = lap.data[20, 20]
    lap = ScalarField(grid, ws[a]).laplace(bc=bc)
    lap_w[a] = lap.data[20, 20]

plt.plot(lap_v)
plt.plot(lap_w)
plt.xlabel("time")
plt.ylabel("Laplace at root")
plt.show()
plt.plot(top_v, label="v")
plt.plot(top_w, label="w")
plt.legend()
plt.xlabel("time")
plt.ylabel("State at root")
plt.show()

In [ ]:
plt.plot(top_v[0:20])
plt.plot(lap_v[0:20])

In [ ]:
vmin = None
vmax = None

In [ ]:
vmin = -2
vmax = 2

This cell actually saves the images to a folder. I watch the animation by opening 0.png and then holding down the RIGHT ARROW key.

In [ ]:

dir = "./fn_images/fixed_z_yamakou"
os.makedirs(dir, exist_ok=True)
save_params(params, f"{dir}/params.json")
for a in tqdm(range(0, 1000, 1)):
    fig = plt.figure()
    fig.set_size_inches((8, 3))
    axs = fig.subplots(1, 2, squeeze=False)
    axes_image = axs[0, 0].imshow(vs[a], vmin=vmin, vmax=vmax)
    plt.colorbar(axes_image, ax=axs[0, 0])
    axes_image = axs[0, 1].imshow(ws[a], vmin=vmin, vmax=vmax)
    plt.colorbar(axes_image, ax=axs[0, 1])
    plt.savefig(f"{dir}/{a}.png")
    plt.close(fig)